In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

project_dir = os.path.dirname(os.path.dirname(os.getcwd()))
if project_dir not in sys.path:
    sys.path.append(project_dir)

from pykrige.ok import OrdinaryKriging

from utils.data import load_data
from utils.tool import prediction_summary

# Prepare Data

In [2]:
os.chdir(project_dir)
train_loader, val_loader, test_loader = load_data()
# change dir back
os.chdir(os.path.join(project_dir, 'baselines/Okriging'))

File1: 3BAGEmnnQ2K4zF49Dkkoxg.csv contains missing hours
File4: 4XEJFVFOS761cvyEjOYf0g.csv contains outliers
File5: 6kzhfU9xTKCUVJMz492l2g.csv contains outliers
File6: 6nBLCf6WT06TOuUExPkBtA.csv contains missing hours
File17: JQ1px-xqQx-xKh3Oa5h9nA.csv contains missing hours
File21: OfAvTbS1SiOjQo4WKSAP9g.csv contains missing hours
File24: R2ebpAblQHylOjteA-2hlQ.csv contains missing hours
File37: jDYxIP2JQL2br5aTIAR7JQ.csv contains outliers
File38: kyRUtBOTTaK7V_-dxOJTwg.csv contains outliers
File45: wSo2iRgjT36eWC4a2joWZg.csv contains outliers


# Interpolation Algorithm

In [3]:
X_m = test_loader.dataset.locations[test_loader.dataset.train_idx, :]
Y_m = test_loader.dataset.readings[:, test_loader.dataset.train_idx]
print(X_m.shape, Y_m.shape)

X_u = test_loader.dataset.locations[test_loader.dataset.test_idx, :]
Y_u = test_loader.dataset.readings[:, test_loader.dataset.test_idx]
print(X_u.shape, Y_u.shape)

n_steps = Y_m.shape[0]
print(n_steps)

torch.Size([28, 2]) torch.Size([2928, 28])
torch.Size([9, 2]) torch.Size([2928, 9])
2928


In [11]:
Y_true = []
Y_pred = []
for i in range(n_steps):
    ok_model = OrdinaryKriging(X_m[:, 0],
                               X_m[:, 1],
                               Y_m[i, :],
                               variogram_model='linear',
                               enable_plotting=False)
    pred, var = ok_model.execute('points', X_u[:, 0], X_u[:, 1])
    target = Y_u[i, :]
    Y_true.append(target)
    Y_pred.append(pred)

Y_true = np.concatenate(Y_true, axis=0)
Y_pred = np.concatenate(Y_pred, axis=0)

prediction_summary(Y_true, Y_pred, True)

RMSE: 6.0446, CVRMSE: 0.3826, MAE: 3.2363, R2: 0.7079
